In [2]:
import torch
from resnet import validate, ResNet, get_data_loader
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, average_precision_score, precision_recall_fscore_support
from matplotlib import pyplot as plt
import numpy as np

In [3]:
day_data_path = '/data/work_amimukh/Journal_IEEE/BDD_Images/TestDay'
night_data_path = '/data/work_amimukh/Journal_IEEE/BDD_Images/TestNight'

In [4]:
res_model = ResNet()

In [34]:
#res_model.load_model('models/run_000/resmodel.pth')
res_model.load_model('model_day_night/run_002/resmodel.pth')

In [35]:
dl_day = get_data_loader(day_data_path, mode='test', batch_size=64)
dl_night = get_data_loader(night_data_path, mode='test', batch_size=64)

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
day_res = []
labels = []
res_model = res_model.to(device)
for img, label in dl_day:
    img = img.to(device)
    label = label.to(device)
    preds = res_model(img)
    day_res.append(preds.cpu().detach().numpy())
    labels.append(label.cpu().detach().numpy())
labels_ = np.hstack(labels)
day_res_ = np.vstack(day_res)
cl_map = dl_day.dataset.class_to_idx
pr = {i:[] for i in cl_map.keys()}
re = {i:[] for i in cl_map.keys()}
plt.figure(figsize=(10, 10))
for label in cl_map.keys():
    label_idx = cl_map[label]
    pr[label], re[label], _ = precision_recall_curve(labels_, day_res_[:,label_idx], pos_label = label_idx)
    #plt.plot(re[label], pr[label], lw=2, label='{}'.format(label))#age_precision_score(labels_, day_res_[:,label_idx], average='micro', pos_label=label_idx)))
    #rec, rec, f = precision_recall_fscore_support(labels_, day_res_[:,label_idx], pos_label=label_idx)
    #rint(label)
   #print('cision:{}, recall:{}, f:{}'.format(prec, rec, f))
#plt.legend()
prec, rec, f, _ = precision_recall_fscore_support(labels_, np.argmax(day_res_, axis=1), average=None)
for idx, l in enumerate(cl_map.keys()):
    print('{}, {}, {}, {}'.format(l, prec[idx], rec[idx], f[idx]))

cars, 0.9656460819927348, 0.8984163769795288, 0.9308188684908208
person, 0.6563032367972743, 0.9350728155339806, 0.7712712712712713
trafficlight, 0.5767605633802817, 0.8882863340563991, 0.699402220324509
trafficsign, 0.9038095238095238, 0.8116313876416507, 0.8552438887011378


<Figure size 720x720 with 0 Axes>

In [38]:
cl_map = dl_night.dataset.class_to_idx
night_res = []
labels = []
res_model = res_model.to(device)
for img, label in dl_night:
    img = img.to(device)
    label = label.to(device)
    preds = res_model(img)
    night_res.append(preds.cpu().detach().numpy())
    labels.append(label.cpu().detach().numpy())
labels_ = np.hstack(labels)
night_res_ = np.vstack(night_res)
pr = {i:[] for i in cl_map.keys()}
re = {i:[] for i in cl_map.keys()}
plt.figure(figsize=(10, 10))
for label in cl_map.keys():
    label_idx = cl_map[label]
    pr[label], re[label], _ = precision_recall_curve(labels_, night_res_[:,label_idx], pos_label = label_idx)
    #plt.plot(re[label], pr[label], lw=2, label='{}'.format(label))#age_precision_score(labels_, day_res_[:,label_idx], average='micro', pos_label=label_idx)))
    #rec, rec, f = precision_recall_fscore_support(labels_, day_res_[:,label_idx], pos_label=label_idx)
    #rint(label)
   #print('cision:{}, recall:{}, f:{}'.format(prec, rec, f))
#plt.legend()
prec, rec, f, _ = precision_recall_fscore_support(labels_, np.argmax(night_res_, axis=1), average=None)
for idx, l in enumerate(cl_map.keys()):
    print('{}, {}, {}, {}'.format(l, prec[idx], rec[idx], f[idx]))

cars, 0.9613691237040193, 0.9032559380838003, 0.9314069487444101
person, 0.5296296296296297, 0.8895800933125972, 0.6639582124201974
trafficlight, 0.5736625514403292, 0.8520782396088019, 0.6856861780619774
trafficsign, 0.9157138190133943, 0.8143521208599651, 0.862063662924804


<Figure size 720x720 with 0 Axes>

In [ ]:
from itertools import cycle
# setup plot details
colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

plt.figure(figsize=(7, 8))
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

lines.append(l)
labels.append('iso-f1 curves')
l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
lines.append(l)
labels.append('micro-average Precision-recall (area = {0:0.2f})'
              ''.format(average_precision["micro"]))

for i, color in zip(range(n_classes), colors):
    l, = plt.plot(recall[i], precision[i], color=color, lw=2)
    lines.append(l)
    labels.append('Precision-recall for class {0} (area = {1:0.2f})'
                  ''.format(i, average_precision[i]))

fig = plt.gcf()
fig.subplots_adjust(bottom=0.25)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Extension of Precision-Recall curve to multi-class')
plt.legend(lines, labels, loc=(0, -.38), prop=dict(size=14))


plt.show()